In [19]:
import requests
import pandas as pd
import os
from urllib.request import urlopen
from bs4 import BeautifulSoup
from bs4 import Comment
import re
import pdb
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

In [38]:
def scrape(player, count, is_test, trim_player_data):
    name = player
    lname = name.split(' ')[1]
    fname = name.split(' ')[0]
    lname = lname.replace("'", "")
    lname = lname.replace('"', "")

    lname = lname[0:5].lower()
    fname = fname[0:2].lower()
    linitial = lname[0:1].lower()

    comm = re.compile("<!--|-->")

    url = "https://www.basketball-reference.com/players/{}/{}{}01.html".format(linitial, lname, fname)
    
    print(name)
    session = requests.Session()
    retry = Retry(connect=3, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    html = session.get(url).content
    soup = BeautifulSoup(re.sub("<!--|-->","", html.decode('utf-8')),"lxml")
    n = soup.find('div', attrs={'id':'meta'})
    player_name = n.find('h1').getText()
    tmp = 2
    if player_name != name:
        print(player_name+" "+ name)
    if name in players:
        count_st = "0"+str(tmp)
        url = "https://www.basketball-reference.com/players/{}/{}{}{}.html".format(linitial, lname, fname, count_st)
        html = requests.get(url).content
        soup = BeautifulSoup(re.sub("<!--|-->","", html.decode('utf-8')),"lxml")
        n = soup.find('div', attrs={'id':'meta'})
        player_name = n.find('h1').getText()
    print(url)
    pposs = soup.find('table', attrs={'id':'per_poss'})
    adv = soup.find('table', attrs={'id':'advanced'})

    pposs_rows = pposs.findAll('tr')[1:-1]
    adv_rows = adv.findAll('tr')[1:-1]
    player_pposs_stats = [[td.getText() for td in pposs_rows[i].findAll('td')]
                for i in range(len(pposs_rows))]
    player_adv_stats = [[td.getText() for td in adv_rows[i].findAll('td')]
                for i in range(len(adv_rows))]

    pposs_headers = [th.getText() for th in pposs.findAll('tr')[0].findAll('th')]
    adv_headers = [th.getText() for th in adv.findAll('tr')[0].findAll('th')]

    # exclude the first column as we will not need the ranking order from Basketball Reference for the analysis
    pposs_headers = pposs_headers[1:]
    adv_headers = adv_headers[1:]
    

    pposs_data = pd.DataFrame(player_pposs_stats, columns = pposs_headers)

    adv_data = pd.DataFrame(player_adv_stats, columns = adv_headers)
    
    allstar = list((""))
    years = list((""))
    for i in range(1, len(pposs_rows) + 1):
        season = [th.getText() for th in pposs.findAll('tr')[i].findAll('th')]
        star = [pposs.findAll('tr')[i].find('span', attrs={'class':'sr_star'})]
        if season[0] == "Career":
            break
        years.append(season[0])
        
        if star[0] != None:
            allstar.append(1)
        else:
            allstar.append(0)
    
    headers = ["Name", "Season", "Pos", "TRB", "AST", "STL", "BLK", "PTS", "TS%", "TRB%", "AST%", 
               "STL%", "BLK%", "USG%", "WS/48", "BPM", "VORP", "Wins", "Seed", "All-Star", "All-NBA"]
    player_data = [[0 for i in range(len(headers))] for j in range(len(years))]
    trim_player_data = list((""))
    skip = 0
    for j in range(len(years)):
        if not is_test  and (years[j] <= "1988-89" or years[j] >= "2018-19" or skip != 0):
            if skip > 0:
                skip -= 1
            continue
        elif is_test and (years[j] < "2018-19" or skip != 0):
            if skip > 0:
                skip -= 1
            continue
        ind = 1
        for i in range(ind):
            player_data[j][i] = name
        for i in range(ind, ind + 1):
            player_data[j][i] = years[j]
        ind += 1
        for i in range(ind, ind + 1):
            stat = headers[i]
            positions = pposs_data[stat]
            cur_pos = positions[j]
            if "G" in cur_pos: 
                player_data[j][i] = "G"
            elif "F" in cur_pos: 
                player_data[j][i] = "F"
            else:
                player_data[j][i] = "C"
        ind += 1
        for i in range(ind, ind + 5):
            stat = headers[i]
            values = pposs_data[stat]
            player_data[j][i] = values[j]
        ind += 5
        for i in range(ind, ind + 9):
            stat = headers[i]
            values = adv_data[stat]
            player_data[j][i] = values[j]
        ind += 9
        for i in range(ind, ind + 1):
            player_team_abbr = pposs_data["Tm"]
            cur_season = years[j]
            player_abbr = player_team_abbr[j]
            player_team = team_abbr.get(player_abbr)
            if player_team == None: 
                skip = 2
                player_team = team_abbr.get(player_team_abbr[j+2])
            player_key = cur_season+" "+player_team
            print(player_key)
            win_seed = all_standings.get(player_key)
            print(win_seed)
            wins = win_seed.split()[0]
            seed = win_seed.split()[1]
            player_data[j][i] = wins
            player_data[j][i+1] = seed
        for i in range(len(headers) - 2, len(headers)-1):
            player_data[j][i] = allstar[j]
        allnbaseasons = allnbaplayers.get(player)
        for i in range(len(headers) - 1, len(headers)):
            if allnbaseasons == None:
                player_data[j][i] = 0
            elif years[j] in allnbaseasons:
                player_data[j][i] = 1
            else:
                player_data[j][i] = 0
        if player_data[j][len(headers) - 2] != 0 or player_data[j][len(headers) - 1] != 0:
            trim_player_data.append(player_data[j])
    trim_data[count] = pd.DataFrame(trim_player_data, columns = headers)
    return trim_data

In [29]:
players = [
    "Anthony Davis",
    "Magic Johnson",
    "David Lee",
    "Rod Strickland",
    "Kevin Willis",
    "Kemba Walker",
    "Joe Johnson",
    "Ray Allen",
    "Eddie Jones",
    "Antoine Walker",
    "Rashard Lewis",
    "Kevin Johnson",
    "Clifford Robinson",
    "Wally Szczerbiak",
    "Tobias Harris",
    "Lou Williams"
]
allnbaurl = "https://www.basketball-reference.com/awards/all_league.html"
allnbahtml = requests.get(allnbaurl)
soup = BeautifulSoup(allnbahtml.content, "lxml")

awards = soup.find('table', attrs={'id':'awards_all_league'})
allnbaplayers = {}

for i in range(1, 119):
    if i % 4 != 0:
        season = awards.findAll('tr')[i].find('th').find('a').getText()
        for j in range(2, 7):
            curplayer = awards.findAll('tr')[i].findAll('td')[j].find('a').getText()
            times = allnbaplayers.get(curplayer)
            if times == None:
                allnbaplayers[curplayer] = season
            else:
                times = season + ", " + times
                allnbaplayers[curplayer] = times

In [30]:
url = "https://en.wikipedia.org/wiki/List_of_NBA_All-Stars"\
html = requests.get(url).content
soup = BeautifulSoup(html,"lxml")
allstar_table = soup.findAll('tr')
allstar_players = [[td.getText() for td in allstar_table[i].findAll('td')]
                for i in range(len(allstar_table))]
for i in range(6, 428):
    starplayer = allstar_players[i][0]
    lastchar = starplayer[-1:]
    starseason = allstar_players[i][2]
    lastseason = starseason[-4:]
    if lastseason > "1989":
        lastcharcount = 0
        if not lastchar.isalpha():
            starplayer = starplayer[:-1]

        times = allnbaplayers.get(starplayer)
        if times == None:
            allnbaplayers[starplayer] = ""

In [31]:
swap_names = {"Anfernee Hardaway":"Penny Hardaway", 
             "Ron Artest": "Metta World Peace",
             "Manu Ginobili": "Manu Ginóbili",
             "Zydrunas Ilgauskas": "Žydrūnas Ilgauskas",
             "BJ Armstrong": "B. J. Armstrong",
             "AC Green": "A. C. Green",
             "Nikola Jokic": "Nikola Jokić",
             "Dangelo Russell": "D'Angelo Russell",
             "Nick VanExel": "Nick Van Exel",
             "Nikola Vucevic": "Nikola Vučević"}
for fixed, wrong in swap_names.items():
    times = allnbaplayers.get(fixed)
    if times == None:
        allnbaplayers[fixed] = allnbaplayers.pop(wrong)
    else:
        allnbaplayers.pop(wrong)
allnbaplayers.pop("Metta World Peace†[d")
allnbaplayers.pop("Hakeem Olajuwon*[b")

''

In [32]:
url = "https://www.basketball-reference.com/leagues/NBA_2019.html"
all_standings = {} 
team_abbr = {}

In [33]:
url = "https://en.wikipedia.org/wiki/Wikipedia:WikiProject_National_Basketball_Association/National_Basketball_Association_team_abbreviations"
html = requests.get(url).content
soup = BeautifulSoup(html,"lxml")
allstar_table = soup.findAll('tr')[1:]
team_abbr_table = [[td.getText() for td in allstar_table[i].findAll('td')]
                for i in range(len(allstar_table))]
for i in range(len(team_abbr_table)):
    cur_team = team_abbr_table[i][1][:-1]
    cur_abbr = team_abbr_table[i][0][:-1]
    #pdb.set_trace()
    team_abbr[cur_abbr] = cur_team
#print(team_abbr)
team_abbr["NOH"] = "New Orleans Hornets"
team_abbr["WSB"] = "Washington Bullets"
team_abbr["SEA"] = "Seattle SuperSonics"
team_abbr["VAN"] = "Vancouver Grizzlies"
team_abbr["CHH"] = "Charlotte Hornets"
team_abbr["CHA"] = "Charlotte Bobcats"
team_abbr["NJN"] = "New Jersey Nets"
team_abbr["NOK"] = "New Orleans/Oklahoma City Hornets"
team_abbr["PHO"] = team_abbr.pop("PHX")
team_abbr["CHO"] = "Charlotte Hornets"
team_abbr["BRK"] = "Brooklyn Nets"

In [34]:
for num_years in range (1990, 2020):
    url = "https://www.basketball-reference.com/leagues/NBA_{}.html".format(num_years)
    html = requests.get(url).content
    soup = BeautifulSoup(html,"lxml")
    e_conf = soup.find('table', attrs={'id':'divs_standings_E'})
    w_conf = soup.find('table', attrs={'id':'divs_standings_W'})
    e_stand = 0
    w_stand = 0
    e_teams = e_conf.findAll('tr', attrs={'class':'full_table'})
    season_year = str(num_years - 1)+"-"+str(num_years)[2:]
    for num_teams in range(1, 19):
        if len(e_conf.findAll('tr')) > num_teams and e_conf.findAll('tr')[num_teams].findAll('td') != []:
            team = e_conf.findAll('a')[e_stand].getText()
            star = e_conf.findAll('tr')[num_teams].find('span', attrs={'class':'seed'}).getText()[1:-2]
            wins = e_conf.findAll('tr')[num_teams].findAll('td')[0].getText()
            if num_years == 1999: 
                wins = str(int(wins)*82/50)
            elif num_years == 2012:
                wins = str(int(wins)*82/66)
            key_standings = season_year+" "+team
            val_standings = wins+" "+star
            all_standings[key_standings] = val_standings
            e_stand += 1
        if len(w_conf.findAll('tr')) > num_teams and w_conf.findAll('tr')[num_teams].findAll('td') != []:
            team = w_conf.findAll('a')[w_stand].getText()
            star = w_conf.findAll('tr')[num_teams].find('span', attrs={'class':'seed'}).getText()[1:-2]
            wins = w_conf.findAll('tr')[num_teams].findAll('td')[0].getText()
            if num_years == 1999: 
                wins = str(int(wins)*82/50)
            elif num_years == 2012:
                wins = str(int(wins)*82/66)
            key_standings = season_year+" "+team
            val_standings = wins+" "+star
            all_standings[key_standings] = val_standings
            w_stand += 1
#print(all_standings)

In [35]:
trim_data = [None for i in range(len(allnbaplayers))]
cur_count = 0
for p in allnbaplayers:
        trim_data = scrape(p, cur_count, False, trim_data)
        cur_count += 1
all_data = trim_data[0]
for i in range(1, cur_count):
    all_data = pd.concat([all_data, trim_data[i]])
#print(all_data)
all_data.to_csv("Data/all_nba_advanced_data.csv")

Giannis Antetokounmpo
https://www.basketball-reference.com/players/a/antetgi01.html
2013-14 Milwaukee Bucks
15 15
2014-15 Milwaukee Bucks
41 6
2015-16 Milwaukee Bucks
33 12
2016-17 Milwaukee Bucks
42 6
2017-18 Milwaukee Bucks
44 7
Paul George
https://www.basketball-reference.com/players/g/georgpa01.html
2010-11 Indiana Pacers
37 8
2011-12 Indiana Pacers
52.18181818181818 3
2012-13 Indiana Pacers
49 3
2013-14 Indiana Pacers
56 1
2014-15 Indiana Pacers
38 9
2015-16 Indiana Pacers
45 7
2016-17 Indiana Pacers
42 7
2017-18 Oklahoma City Thunder
48 4
James Harden
https://www.basketball-reference.com/players/h/hardeja01.html
2009-10 Oklahoma City Thunder
50 8
2010-11 Oklahoma City Thunder
55 4
2011-12 Oklahoma City Thunder
58.39393939393939 2
2012-13 Houston Rockets
45 8
2013-14 Houston Rockets
54 4
2014-15 Houston Rockets
56 2
2015-16 Houston Rockets
41 8
2016-17 Houston Rockets
55 3
2017-18 Houston Rockets
65 1
Stephen Curry
https://www.basketball-reference.com/players/c/curryst01.html
2009

https://www.basketball-reference.com/players/p/paulch01.html
2005-06 New Orleans/Oklahoma City Hornets
38 10
2006-07 New Orleans/Oklahoma City Hornets
39 10
2007-08 New Orleans Hornets
56 2
2008-09 New Orleans Hornets
49 7
2009-10 New Orleans Hornets
37 11
2010-11 New Orleans Hornets
46 7
2011-12 Los Angeles Clippers
49.696969696969695 5
2012-13 Los Angeles Clippers
56 4
2013-14 Los Angeles Clippers
57 3
2014-15 Los Angeles Clippers
56 2
2015-16 Los Angeles Clippers
53 4
2016-17 Los Angeles Clippers
51 4
2017-18 Houston Rockets
65 1
Andre Drummond
https://www.basketball-reference.com/players/d/drumman01.html
2012-13 Detroit Pistons
29 11
2013-14 Detroit Pistons
29 11
2014-15 Detroit Pistons
32 12
2015-16 Detroit Pistons
44 8
2016-17 Detroit Pistons
37 10
2017-18 Detroit Pistons
39 9
Klay Thompson
https://www.basketball-reference.com/players/t/thompkl01.html
2011-12 Golden State Warriors
28.575757575757574 13
2012-13 Golden State Warriors
47 6
2013-14 Golden State Warriors
51 6
2014-15 

https://www.basketball-reference.com/players/b/bynuman01.html
2005-06 Los Angeles Lakers
45 7
2006-07 Los Angeles Lakers
42 7
2007-08 Los Angeles Lakers
57 1
2008-09 Los Angeles Lakers
65 1
2009-10 Los Angeles Lakers
57 1
2010-11 Los Angeles Lakers
57 2
2011-12 Los Angeles Lakers
50.93939393939394 3
2013-14 Indiana Pacers
56 1
Tyson Chandler
https://www.basketball-reference.com/players/c/chandty01.html
2001-02 Chicago Bulls
21 15
2002-03 Chicago Bulls
30 12
2003-04 Chicago Bulls
23 14
2004-05 Chicago Bulls
47 4
2005-06 Chicago Bulls
41 7
2006-07 New Orleans/Oklahoma City Hornets
39 10
2007-08 New Orleans Hornets
56 2
2008-09 New Orleans Hornets
49 7
2009-10 Charlotte Bobcats
44 7
2010-11 Dallas Mavericks
57 3
2011-12 New York Knicks
44.72727272727273 7
2012-13 New York Knicks
54 2
2013-14 New York Knicks
37 9
2014-15 Dallas Mavericks
50 7
2015-16 Phoenix Suns
23 14
2016-17 Phoenix Suns
24 15
2017-18 Phoenix Suns
21 15
Dirk Nowitzki
https://www.basketball-reference.com/players/n/nowitdi

https://www.basketball-reference.com/players/g/garneke01.html
1995-96 Minnesota Timberwolves
26 12
1996-97 Minnesota Timberwolves
40 6
1997-98 Minnesota Timberwolves
45 7
1998-99 Minnesota Timberwolves
41.0 8
1999-00 Minnesota Timberwolves
50 6
2000-01 Minnesota Timberwolves
47 8
2001-02 Minnesota Timberwolves
50 5
2002-03 Minnesota Timberwolves
51 4
2003-04 Minnesota Timberwolves
58 1
2004-05 Minnesota Timberwolves
44 9
2005-06 Minnesota Timberwolves
33 14
2006-07 Minnesota Timberwolves
32 13
2007-08 Boston Celtics
66 1
2008-09 Boston Celtics
62 2
2009-10 Boston Celtics
50 4
2010-11 Boston Celtics
56 3
2011-12 Boston Celtics
48.45454545454545 4
2012-13 Boston Celtics
41 7
2013-14 Brooklyn Nets
44 5
2014-15 Minnesota Timberwolves
16 15
2015-16 Minnesota Timberwolves
29 13
Carlos Boozer
https://www.basketball-reference.com/players/b/boozeca01.html
2002-03 Cleveland Cavaliers
17 15
2003-04 Cleveland Cavaliers
35 9
2004-05 Utah Jazz
26 14
2005-06 Utah Jazz
41 9
2006-07 Utah Jazz
51 4
2007

https://www.basketball-reference.com/players/r/reddmi01.html
2000-01 Milwaukee Bucks
52 2
2001-02 Milwaukee Bucks
41 9
2002-03 Milwaukee Bucks
42 7
2003-04 Milwaukee Bucks
41 6
2004-05 Milwaukee Bucks
30 13
2005-06 Milwaukee Bucks
40 8
2006-07 Milwaukee Bucks
28 14
2007-08 Milwaukee Bucks
26 13
2008-09 Milwaukee Bucks
34 12
2009-10 Milwaukee Bucks
46 6
2010-11 Milwaukee Bucks
35 9
2011-12 Phoenix Suns
41.0 10
Chris Webber
https://www.basketball-reference.com/players/w/webbech01.html
1993-94 Golden State Warriors
50 6
1994-95 Washington Bullets
21 14
1995-96 Washington Bullets
39 10
1996-97 Washington Bullets
44 8
1997-98 Washington Wizards
42 9
1998-99 Sacramento Kings
44.28 7
1999-00 Sacramento Kings
44 8
2000-01 Sacramento Kings
55 3
2001-02 Sacramento Kings
61 1
2002-03 Sacramento Kings
59 2
2003-04 Sacramento Kings
55 4
2004-05 Philadelphia 76ers
43 7
2005-06 Philadelphia 76ers
38 9
2006-07 Detroit Pistons
53 1
2007-08 Golden State Warriors
48 9
Jamal Mashburn
https://www.basketbal

...

https://www.basketball-reference.com/players/c/colemde01.html
1990-91 New Jersey Nets
26 11
1991-92 New Jersey Nets
40 6
1992-93 New Jersey Nets
43 6
1993-94 New Jersey Nets
45 7
1994-95 New Jersey Nets
30 11
1995-96 Philadelphia 76ers
18 15
1996-97 Philadelphia 76ers
22 14
1997-98 Philadelphia 76ers
31 14
1998-99 Charlotte Hornets
42.64 9
1999-00 Charlotte Hornets
49 4
2000-01 Charlotte Hornets
46 6
2001-02 Philadelphia 76ers
43 6
2002-03 Philadelphia 76ers
48 4
2003-04 Philadelphia 76ers
33 11
2004-05 Detroit Pistons
54 2
Dominique Wilkins
https://www.basketball-reference.com/players/w/wilkido01.html
1989-90 Atlanta Hawks
41 9
1990-91 Atlanta Hawks
43 6
1991-92 Atlanta Hawks
38 9
1992-93 Atlanta Hawks
43 7
1993-94 Los Angeles Clippers
27 11
1994-95 Boston Celtics
35 8
1996-97 San Antonio Spurs
20 13
1998-99 Orlando Magic
54.12 3
Mark Price
https://www.basketball-reference.com/players/p/pricema01.html
1989-90 Cleveland Cavaliers
42 7
1990-91 Cleveland Cavaliers
33 9
1991-92 Cleveland 

https://www.basketball-reference.com/players/b/bealbr01.html
2012-13 Washington Wizards
29 12
2013-14 Washington Wizards
44 5
2014-15 Washington Wizards
46 5
2015-16 Washington Wizards
41 10
2016-17 Washington Wizards
49 4
2017-18 Washington Wizards
43 8
Terrell Brandon
https://www.basketball-reference.com/players/b/brandte01.html
1991-92 Cleveland Cavaliers
57 3
1992-93 Cleveland Cavaliers
54 3
1993-94 Cleveland Cavaliers
47 6
1994-95 Cleveland Cavaliers
43 6
1995-96 Cleveland Cavaliers
47 4
1996-97 Cleveland Cavaliers
42 9
1997-98 Milwaukee Bucks
36 13
1998-99 Minnesota Timberwolves
41.0 8
1999-00 Minnesota Timberwolves
50 6
2000-01 Minnesota Timberwolves
47 8
2001-02 Minnesota Timberwolves
50 5
Caron Butler
https://www.basketball-reference.com/players/b/butleca01.html
2002-03 Miami Heat
25 13
2003-04 Miami Heat
42 4
2004-05 Los Angeles Lakers
34 11
2005-06 Washington Wizards
42 5
2006-07 Washington Wizards
41 7
2007-08 Washington Wizards
43 5
2008-09 Washington Wizards
19 15
2009-10

2012-13 Indiana Pacers
49 3
2013-14 Indiana Pacers
56 1
2014-15 Indiana Pacers
38 9
2015-16 San Antonio Spurs
67 2
2016-17 Golden State Warriors
67 1
2017-18 Golden State Warriors
58 2
Shareef Abdur-Rahim
https://www.basketball-reference.com/players/a/abdursh01.html
1996-97 Vancouver Grizzlies
14 14
1997-98 Vancouver Grizzlies
19 11
1998-99 Vancouver Grizzlies
13.12 14
1999-00 Vancouver Grizzlies
22 12
2000-01 Vancouver Grizzlies
23 13
2001-02 Atlanta Hawks
33 12
2002-03 Atlanta Hawks
35 11
2003-04 Portland Trail Blazers
41 10
2004-05 Portland Trail Blazers
27 13
2005-06 Sacramento Kings
44 8
2006-07 Sacramento Kings
33 11
2007-08 Sacramento Kings
38 11
Michael Adams
https://www.basketball-reference.com/players/a/adamsmi01.html
1989-90 Denver Nuggets
43 7
1990-91 Denver Nuggets
20 14
1991-92 Washington Bullets
25 13
1992-93 Washington Bullets
22 14
1993-94 Washington Bullets
24 12
1994-95 Charlotte Hornets
50 4
1995-96 Charlotte Hornets
41 9
Kenny Anderson
https://www.basketball-refere

https://www.basketball-reference.com/players/h/hornaje01.html
1989-90 Phoenix Suns
54 5
1990-91 Phoenix Suns
55 4
1991-92 Phoenix Suns
53 4
1992-93 Philadelphia 76ers
26 13
1993-94 Utah Jazz
53 5
1994-95 Utah Jazz
60 3
1995-96 Utah Jazz
55 3
1996-97 Utah Jazz
64 1
1997-98 Utah Jazz
62 1
1998-99 Utah Jazz
60.68 2
1999-00 Utah Jazz
55 2
Josh Howard
https://www.basketball-reference.com/players/h/howarjo01.html
2003-04 Dallas Mavericks
52 5
2004-05 Dallas Mavericks
58 4
2005-06 Dallas Mavericks
60 4
2006-07 Dallas Mavericks
67 1
2007-08 Dallas Mavericks
51 7
2008-09 Dallas Mavericks
50 6
2009-10 Washington Wizards
26 14
2010-11 Washington Wizards
23 13
2011-12 Utah Jazz
44.72727272727273 8
2012-13 Minnesota Timberwolves
31 12
Andre Iguodala
https://www.basketball-reference.com/players/i/iguodan01.html
2004-05 Philadelphia 76ers
43 7
2005-06 Philadelphia 76ers
38 9
2006-07 Philadelphia 76ers
35 10
2007-08 Philadelphia 76ers
40 7
2008-09 Philadelphia 76ers
41 6
2009-10 Philadelphia 76ers
27 

https://www.basketball-reference.com/players/s/simmobe01.html
2017-18 Philadelphia 76ers
52 3
Steve Smith
https://www.basketball-reference.com/players/s/smithst01.html
1991-92 Miami Heat
38 8
1992-93 Miami Heat
36 11
1993-94 Miami Heat
42 8
1994-95 Atlanta Hawks
42 7
1995-96 Atlanta Hawks
46 6
1996-97 Atlanta Hawks
56 4
1997-98 Atlanta Hawks
50 5
1998-99 Atlanta Hawks
50.84 4
1999-00 Portland Trail Blazers
59 3
2000-01 Portland Trail Blazers
50 7
2001-02 San Antonio Spurs
58 2
2002-03 San Antonio Spurs
60 1
2003-04 New Orleans Hornets
41 5
2004-05 Miami Heat
59 1
Rik Smits
https://www.basketball-reference.com/players/s/smitsri01.html
1989-90 Indiana Pacers
42 8
1990-91 Indiana Pacers
41 7
1991-92 Indiana Pacers
40 7
1992-93 Indiana Pacers
41 8
1993-94 Indiana Pacers
47 5
1994-95 Indiana Pacers
52 2
1995-96 Indiana Pacers
52 3
1996-97 Indiana Pacers
39 10
1997-98 Indiana Pacers
58 3
1998-99 Indiana Pacers
54.12 2
1999-00 Indiana Pacers
56 1
John Starks
https://www.basketball-reference.c

In [40]:
test_players = ["James Harden", "Giannis Antetokounmpo", "Stephen Curry", "LeBron James", 
              "Kawhi Leonard", "Anthony Davis", "Kevin Durant", "Nikola Jokic", "Paul George", "Kyrie Irving",
              "Joel Embiid", "Rudy Gobert", "Damian Lillard", "Karl-Anthony Towns", "Blake Griffin",
              "Russell Westbrook", "Jimmy Butler", "Kemba Walker", "Ben Simmons", "Kyle Lowry",
              "Bradley Beal", "Mike Conley", "Jrue Holiday", "LaMarcus Aldridge", "Nikola Vucevic",
              "Victor Oladipo", "Klay Thompson", "Khris Middleton", "DAngelo Russell"]

In [39]:
test_trim_data = [None for i in range(len(test_players))]
cur_test_count = 0
for testplayer in test_players:
    test_trim_data = scrape(testplayer, cur_test_count, True, test_trim_data)
    cur_test_count += 1
all_test_data = test_trim_data[0]
for i in range(1, cur_test_count):
    all_test_data = pd.concat([all_test_data, test_trim_data[i]])
print(all_test_data)
all_test_data.to_csv("test_nba_advanced_data.csv")

James Harden
https://www.basketball-reference.com/players/h/hardeja01.html
2018-19 Houston Rockets
53 4
Giannis Antetokounmpo
https://www.basketball-reference.com/players/a/antetgi01.html
2018-19 Milwaukee Bucks
60 1
Stephen Curry
https://www.basketball-reference.com/players/c/curryst01.html
2018-19 Golden State Warriors
57 1
LeBron James
https://www.basketball-reference.com/players/j/jamesle01.html
2018-19 Los Angeles Lakers
37 10
Kevin Durant
https://www.basketball-reference.com/players/d/duranke01.html
2018-19 Golden State Warriors
57 1
Kawhi Leonard
https://www.basketball-reference.com/players/l/leonaka01.html
2018-19 Toronto Raptors
58 2
Anthony Davis
Antonio Davis Anthony Davis
https://www.basketball-reference.com/players/d/davisan02.html
2018-19 New Orleans Pelicans
33 13
Kevin Durant
https://www.basketball-reference.com/players/d/duranke01.html
2018-19 Golden State Warriors
57 1
Nikola Jokic
Nikola Jokić Nikola Jokic
https://www.basketball-reference.com/players/j/jokicni01.html